In [1]:
from netcdf_segy.segy2netcdf import segy2netcdf
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import glob
import os
from pathlib import Path
import cmocean
import datetime
import numpy as np
import matplotlib.dates as mdates

## JR127 SBP

indir='x:/SONAR ARCHIVE/Multibeam Data/JR127 SBP/*.seg'
fles=glob.glob(indir)

outdir='x:/SONAR ARCHIVE/Multibeam Data/JR127 SBP/Lewis-segy-conversion'
os.makedirs(outdir, exist_ok=True) 

for i in fles:
    if i=='x:/SONAR ARCHIVE/Multibeam Data/JR127 SBP\\050916201227.0.seg':
        print('ignoring '+ i)
    else:
            
        segin=i
        fname=Path(segin).stem
        ncout=os.path.join(outdir,fname+'.nc')
        imout=os.path.join(outdir,fname+'.png')

        segy2netcdf(segin, ncout)
        shots = xr.open_dataset(ncout)
        
        from datetime import date
        tdata=[]

        fyr='20'+fname[0:2]
        fmn=fname[2:4]
        fdy=fname[4:6]

        hday=shots.HourOfDay.values
        mhour=shots.MinuteOfHour.values
        smin=shots.SecondOfMinute.values

        for t in range (0,len(hday)):
            d=datetime.datetime(int(fyr),int(fmn),int(fdy),hour=hday[t], minute=mhour[t], second=smin[t])
            td=d.isoformat()
            tdata.append(td)
        
        shots.assign(dtime=tdata)
        
        samplenumber=shots.coords['SampleNumber'].values
        sample=shots.Samples.values
        x = xr.Dataset(
            {
                "sample": (
                    ("dtime","samplenumber" ),
                    sample,
                )
            },
            coords={"samplenumber": samplenumber, "dtime": tdata},
        )

        shots=shots.assign(x)
        fig, ax = plt.subplots(figsize=(10, 8))
        shots.sample.plot(x="dtime",y="samplenumber",cmap='RdGy',vmin=-2000., vmax=2000.,ax=ax)
        # Make ticks on occurrences of each month:
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=1200))
        # Get only the month to show in the x-axis:
        # ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
        # Rotating X-axis labels
        plt.xticks(rotation = 90)
        plt.gca().invert_yaxis()
        plt.grid(axis='x')
        plt.title(i)
        plt.savefig(imout)
        plt.close()
        shots.close()

ignoring x:/SONAR ARCHIVE/Multibeam Data/JR127 SBP\050916201227.0.seg
